#Imports

We start by importing essential libraries:
- `ray` is used for distributed computing and managing actors.
- `bitcoin_utils` contains the core API functions we want to test.
- `pandas` and others help us manage and inspect data.

In [1]:
!pip install -r requirements.txt
import ray
from bitcoin_utils import (
    fetch_bitcoin_price,
    PriceProcessor,
    run_price_stream,
    load_csv_to_actor,
    fetch_hourly_btc_yfinance
)
import pandas as pd

2025-05-17 17:54:31,584	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.23gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-05-17 17:54:31,705	INFO worker.py:1752 -- Started a local Ray instance.


#Starting Ray Cluster

We initialize a local Ray instance so we can use remote actors and functions.
This is required before using any `@ray.remote` classes or methods.

In [2]:
ray.shutdown()
ray.init(ignore_reinit_error=True, include_dashboard=False)

2025-05-17 17:54:35,016	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.23gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-05-17 17:54:36,148	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.10.0


#Fetch Single Price from CoinGecko

This uses `fetch_bitcoin_price.remote()` to get the current BTC price and timestamp from the CoinGecko API.
Ray’s `ray.get()` retrieves the result from the remote function.

In [3]:
timestamp, price = ray.get(fetch_bitcoin_price.remote())
print(f"Fetched Real-Time Price: ${price} at timestamp {timestamp}")

Fetched Real-Time Price: $103239 at timestamp 1747504476.7452488


#Start a Ray Actor

Here, we create a Ray actor (`PriceProcessor`) which holds the BTC price data in memory.
It supports moving averages, volatility, filtering, and more.

In [4]:
processor = PriceProcessor.remote()


#Add Price to Actor

This sends the current price + timestamp to the actor's internal store.

In [5]:
ray.get(processor.add_price.remote(timestamp, price))

1

#Get Data Back

We retrieve the current price list from the actor in a human-readable timestamp format.

In [6]:
data = ray.get(processor.get_data_with_readable_time.remote())
print("Stored Prices in Actor:", data)

Stored Prices in Actor: [{'timestamp': '2025-05-17 17:54:36', 'price': 103239}]


#Moving Average

This computes a moving average of stored prices over the last `window` entries.
It's useful for smoothing out short-term fluctuations.


In [7]:
print("Computing Moving Average...")
moving_avg = ray.get(processor.compute_moving_average.remote(window=1))
print("Moving Avg Output:", moving_avg)

Computing Moving Average...
Moving Avg Output: [{'timestamp': 1747504476.7452488, 'price': 103239, 'moving_avg': 103239.0}]


#Percentage Change (if more than 1 point)

We add another price point and compute the percentage change between consecutive price entries.


In [8]:
ray.get(processor.add_price.remote(timestamp + 60, price + 50))
pct_change = ray.get(processor.compute_percentage_changes.remote())
print("Percentage Changes:", pct_change)

Percentage Changes: [{'timestamp': '2025-05-17 17:55:36', 'price': 103289, 'percent_change': 0.0484}]



#Filtered Results

This filters and returns price records where the BTC value exceeds the given threshold.
Useful for tracking spikes or triggers.

In [9]:
print("Prices above $20000:")
print(ray.get(processor.filter_prices_above.remote(20000)))

Prices above $20000:
[{'timestamp': '2025-05-17 17:54:36', 'price': 103239}, {'timestamp': '2025-05-17 17:55:36', 'price': 103289}]


#Volatility

Volatility is computed over a sliding window, representing how much price fluctuates.
Higher values mean more instability in the market.

In [10]:
volatility = ray.get(processor.compute_volatility.remote(window=2))
print("Volatility Sample:", volatility)

Volatility Sample: []


# Fetch historical with yfinance

This function pulls recent Bitcoin OHLC (open-high-low-close) prices using the `yfinance` API.
We print the first few entries to verify it worked.

In [11]:
btc_hist = fetch_hourly_btc_yfinance(period="1d", interval="5m")
print("BTC YFinance Snapshot:")
print(btc_hist.head())

Fetching BTC-USD data for the past 1d at 5m intervals...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

BTC YFinance Snapshot:
Price                   Datetime          Close           High            Low  \
Ticker                                  BTC-USD        BTC-USD        BTC-USD   
0      2025-05-17 00:00:00+00:00  103463.453125  103495.781250  103458.617188   
1      2025-05-17 00:05:00+00:00  103520.117188  103520.117188  103458.593750   
2      2025-05-17 00:10:00+00:00  103559.625000  103559.625000  103467.015625   
3      2025-05-17 00:15:00+00:00  103469.312500  103557.039062  103461.015625   
4      2025-05-17 00:20:00+00:00  103421.101562  103428.703125  103413.804688   

Price            Open      Volume  
Ticker        BTC-USD     BTC-USD  
0       103495.781250   834117632  
1       103484.453125    46313472  
2       103525.492188   230592512  
3       103557.039062  1133232128  
4       103428.703125    74170368  


#Done

Always good practice to shut down the Ray runtime when you're done.

In [12]:
ray.shutdown()